# Установка библиотек

In [2]:
import pandas as pd #для работы с таблицами

Для автоматического применения статистического критерия (теста) используется модуль `stats` библиотеки `scipy`.

Этот модуль содержит большое количество вероятностных распределений, корреляционные функции и статистические тесты и многое другое.

Также активно используется библиотека `statsmodels`

Для установки библиотеки `scipy` и `statsmodels` можно воспользоваться менеджером пакетов `pip`.

In [3]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



# Загрузка данных

Для скачивания данных используем команду `!wget`. 

Данные появятся в локальной директории.

Мы будем использовать информацию о диаметре пицц разных производителей.

In [5]:
%pip install wget

Note: you may need to restart the kernel to use updated packages.


In [6]:
import wget

In [7]:
!wget https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2023-01-17 16:40:37--  https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv
����������� raw.githubusercontent.com... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
��������������� ���������� � raw.githubusercontent.com|185.199.110.133|:443... ���������� �����������.
���������� ���������� ���������� SSL.


Загрузим данные в память ноутбука.

In [8]:
data = pd.read_csv('pizzas.csv')

In [9]:
data.rename(columns={'Making Unit 1': 'Пиццерия №1', 'Making Unit 2': 'Пиццерия №2'}, inplace=True)

In [10]:
data.head()

,Пиццерия №1,Пиццерия №2
0,6.8090,6.7703
1,6.4376,7.5093
2,6.9157,6.7300
3,7.3012,6.7878
4,7.4488,7.1522


# Проверка данных на нормальность


In [11]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [12]:
alpha = 0.05

## Тест Шапиро-Уилка

In [13]:
from scipy.stats import shapiro

In [14]:
_, p = shapiro(data)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.204
Данные распределены нормально


## Тест Д’Агостино

In [15]:
from scipy.stats import normaltest

In [16]:
_, p = normaltest(data)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.251
Данные распределены нормально


# Независимый T-тест 

In [17]:
from scipy.stats import ttest_ind

In [18]:
H0 = 'Нет значимой разницы между диаметрами пиццы в разных пиццериях.'
Ha = 'Есть значимая разница между диаметрами пиццы в разных пиццериях.'

Зависимая переменная (диаметр пиццы) является количественной. Группы происходят из разных совокупностей. Следовательно, мы используем независимый T-тест.

In [19]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(data['Пиццерия №1'], data['Пиццерия №2'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(data)


*** Результаты независимого T-теста ***
0.47 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет значимой разницы между диаметрами пиццы в разных пиццериях.


-----

# Корреляция Спирмана 

In [20]:
from numpy.random import rand
from scipy.stats import spearmanr

data1 = rand(1000) * 20
data2 = data1 + (rand(1000) * 10)


corr, p = spearmanr(data1, data2)

In [21]:
print(corr,p)

0.8964097764097764 0.0


In [22]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.0 <= 0.05. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.


# ANOVA тест

In [23]:
from scipy.stats import f_oneway

Однофакторный дисперсионный анализ ANOVA проверяет нулевую гипотезу о том, что две или более групп имеют одинаковое среднее значение генеральной совокупности.

В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

In [24]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

In [25]:
_, p = f_oneway(petersburg, magadan, tvarminne)

In [26]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в трех разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в трех разных местах.'

In [27]:
if p>alpha:
  print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
  print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.007870900547143827 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в трех разных местах.


_____

# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [28]:
# имопртируем модуль Пандас
import pandas as pd
  
# пребразуем список petersburg в dataframe df1
df1 = pd.DataFrame(
  {"СПб": petersburg},
  index=[1, 2, 3, 4, 5, 6, 7])
  
# пребразуем список magadan в dataframe df2
df2 = pd.DataFrame(
  {"Магадан": magadan},
  index=[1, 2, 3, 4, 5, 6, 7, 8])

frames = [df1, df2]
  
# объединим 2 фрейма данных
df = pd.concat(frames, join='outer', axis=1)
display(df)

,СПб,Магадан
1,0.0974,0.1033
2,0.1352,0.0915
3,0.0817,0.0781
4,0.1016,0.0685
5,0.0968,0.0677
6,0.1064,0.0697
7,0.1050,0.0764
8,NaN,0.0689


In [29]:
# заполним значения NaN в столбце средним значением
df['СПб'] = df['СПб']. fillna(df['СПб'].mean())
display(df)

,СПб,Магадан
1,0.097400,0.1033
2,0.135200,0.0915
3,0.081700,0.0781
4,0.101600,0.0685
5,0.096800,0.0677
6,0.106400,0.0697
7,0.105000,0.0764
8,0.103443,0.0689


In [30]:
# на всякий случай преобразуем наши данные в тип float
df = df.astype({"СПб":'float', "Магадан":'float'}) 

In [31]:
# найдём разницу средних значений столбцов
mussel_mean = (df['СПб'].mean() - df['Магадан'].mean ())
print(mussel_mean)

0.025430357142857143


## Проверка данных на нормальность

In [32]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

In [33]:
# Установим уровень значимости
alpha = 0.05

In [34]:
# Тест Шапиро-Уилка
from scipy.stats import shapiro

In [35]:
_, p = shapiro(df)
print('p=%.3f' % p)

# Интерпретация 
if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.100
Данные распределены нормально


In [36]:
# Тест Д’Агостино
from scipy.stats import normaltest

In [37]:
_, p = normaltest(df)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.042
Данные распределены нормально


c:\Users\Александр\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


## Выполним тест на корреляцию.
Так как у нас признаки непрерывные нормально распределённые, то принимает тест Пирсона.

In [38]:
# определим индекс корреляции
print(df['СПб'].corr(df['Магадан']))

0.24455753997284527


In [39]:
# преобразуем столбец «СПб» в DataFrame в список
SPb = df['СПб']. tolist ()
SPb

[0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105, 0.10344285714285714]

In [40]:
# преобразуем столбец «Магадан» в DataFrame в список
Mag = df['Магадан']. tolist ()
Mag

[0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

In [43]:
from scipy.stats import pearsonr

# расчет коэффициента корреляции и значения p между SPb и Mag
corr, p = pearsonr(SPb, Mag)
print(corr,p)

0.24455753997284532 0.5594098062056254


In [44]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.5594098062056254 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.


Определим какова разница в среднем размере мидий в зависимости от города-производителя.
Независимая переменная (город) категориальная: 1 переменная. Зависимая переменная (размер мидий) является количественной. Группы происходят из разных совокупностей. Сравнение проводиться 2-х групп, выборка меньше 30. Следовательно, применяем Т-тест (t-критерий).

## Независимый T-тест

In [127]:
H0 = 'Нет значимой разницы между средними размерами мидий в разных городах.'
Ha = 'Есть значимая разница между средними размерами мидий в разных гродах.'

In [128]:
from scipy.stats import ttest_ind

In [129]:
# проведём расчёт, используя независимый T-тест.
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['СПб'], df['Магадан'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(df)


*** Результаты независимого T-теста ***
0.0 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средними размерами мидий в разных гродах.


# Бонус

Скачаем датасет, содержащий информацию по давлению 120 человек.



In [130]:
%pip install wget

Note: you may need to restart the kernel to use updated packages.


In [131]:
import wget

In [132]:
!wget https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2023-01-17 14:03:39--  https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv
����������� raw.githubusercontent.com... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
��������������� ���������� � raw.githubusercontent.com|185.199.108.133|:443... ���������� �����������.
���������� ���������� ���������� SSL.


In [135]:
data = pd.read_csv('blood_pressure.csv')

In [136]:
print('Размер выборки: ', data.shape)

Размер выборки:  (120, 5)


In [137]:
data.head()

,patient,sex,agegrp,bp_before,bp_after
0,1,Male,30-45,143,153
1,2,Male,30-45,163,170
2,3,Male,30-45,153,168
3,4,Male,30-45,153,142
4,5,Male,30-45,146,141


## Z-тест

Проведем Z-тест для проверки двух независимых групп данных на равенство средних значений выборок.

In [138]:
from statsmodels.stats import weightstats

In [139]:
_ ,p = weightstats.ztest(data['bp_before'], x2=data['bp_after'], value=0,alternative='two-sided')
print(float(p))


if p < 0.05:
    print("Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу об отсутсвии различий между данными.")

0.002162306611369422
Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.


## Z-тест на пропорцию

Z-критерий одной пропорции используется для сравнения наблюдаемой пропорции с теоретической.

В этом тесте используются следующие нулевые гипотезы:

$H_0: p = p_0$ (доля мужчин, предоставивших данные о своем давлении, равна гипотетической пропорции $p_0$)

Альтернативная гипотеза может быть двусторонней, левосторонней или правосторонней:

$H_1 (двусторонний): p ≠ p_0$ (доля мужчин не равна некоторому гипотетическому значению $p_0$)
$H_1 (левосторонний): p < p_0$ (доля мужчин меньше некоторого гипотетического значения $p_0$)
$H_1 (правосторонний): p > p_0$ (доля мужчин больше некоторого гипотетического значения $p_0$)

Предположим, что доля мужчин в нашем датачете равна 40%.

$p_0$: гипотетическая доля мужчин = 0.40

$x$: число мужчин в выборке мужчин: `len(data[data.sex == 'Male'])`

$n$: размер выборки = `len(data)`

Покажем, как использовать функцию `proportions_ztest` для выполнения  **z-теста**:

In [140]:
p_0 = 0.4
n = len(data)
x = len(data[data.sex == 'Male'])

print(n, x)

120 60


In [141]:
from statsmodels.stats.proportion import proportions_ztest

#perform one proportion z-test
_, p = proportions_ztest(count=x, nobs=n, value=p_0)

In [142]:
if p < 0.05:
    print("Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу.")

Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.
